# Task 3 - Epithelial to Mesenchymal (EMT) score from the Hallmark EMT or Epithelial to Mesenchymal Plasticity
This notebook analyses the Epithelial to Mesenchymal (EMT) score from the Hallmark EMT or Epithelial to Mesenchymal Plasticity score from [https://www.science.org/doi/10.1126/sciadv.abi7640](https://www.science.org/doi/10.1126/sciadv.abi7640) as well as HALLMARK.

This notebook is based on the single-cell best practices book (https://github.com/theislab/single-cell-best-practices/). For further explanations of the applied methods please refer to above source.

# Gene set enrichment and pathway analysis 

### Prepare and explore the data

In [1]:
import os
os.environ['R_HOME'] = '/home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R'

In [2]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import anndata as ad
import gdown
import anndata2ri
import rpy2
from rpy2.robjects import r
import random
import session_info

anndata2ri.activate()

In [3]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))

In [4]:
%load_ext rpy2.ipython

In [5]:
!python -m rpy2.situation

rpy2 version:
3.5.1
Python version:
3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
Looking for R's HOME:
    Environment variable R_HOME: /home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R
    Calling `R RHOME`: /home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R
    Environment variable R_LIBS_USER: ~/R/x86_64-conda-linux-gnu-library/4.0
R's additions to LD_LIBRARY_PATH:
/home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R/lib/:/home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R/lib/
R version:
    In the PATH: R version 4.0.5 (2021-03-31) -- "Shake and Throw"
    Loading R library from rpy2: OK
Additional directories to load R packages from:

C extension compilation:
  include:
  ['/home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'rt', 'dl', 'm', 'iconv', 'icuuc', 'icui18n']
  library_dirs:
  ['/home/icb/till.richter/anaconda3/envs/

In [6]:
%%R
suppressPackageStartupMessages({
    library(SingleCellExperiment)
    library(fgsea)
    library(AUCell)
    library(ggplot2)
})

In [7]:
adata_read = sc.read_h5ad('../data/adata/processed_adata3.h5ad')
adata_read.obs

,sample_id,sample_nr,donor,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,log_counts,n_counts,n_genes,size_factors,louvain_r0.5
AAACCCAAGCTGCCTG-1-1,16992-2D,1,mouse,3113,8.043663,9743.0,9.184407,34.014164,42.297034,51.021246,64.087037,1352.0,7.210080,13.876629,9.184304,9743.0,3113,2.360472,0
AAACCCAAGGTCATAA-1-1,16992-2D,1,mouse,2564,7.849714,7168.0,8.877522,27.734375,38.085938,48.660714,63.825335,507.0,6.230482,7.073103,8.877382,7168.0,2564,1.786558,1
AAACCCAAGTACCCTA-1-1,16992-2D,1,mouse,2819,7.944492,8436.0,9.040382,27.750119,38.205311,47.878141,62.849692,630.0,6.447306,7.467994,9.040263,8436.0,2819,2.110656,1
AAACCCACAAGGTCTT-1-1,16992-2D,1,mouse,1932,7.566828,4536.0,8.420021,26.410935,36.331570,47.266314,64.285714,277.0,5.627621,6.106702,8.419801,4536.0,1932,1.236708,0
AAACCCACAGGAGGTT-1-1,16992-2D,1,mouse,2930,7.983099,8959.0,9.100526,29.679652,38.910593,48.018752,62.808349,604.0,6.405229,6.741824,9.100414,8959.0,2930,2.218654,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTCTAGGTT-1-1,16992-2D,1,mouse,2192,7.693026,5087.0,8.534640,25.122862,34.499705,44.584234,60.723413,301.0,5.710427,5.917044,8.534444,5087.0,2192,1.391775,1
TTTGTTGGTGGACTGA-1-1,16992-2D,1,mouse,2301,7.741534,4901.0,8.497398,22.444399,31.034483,41.154866,57.702510,258.0,5.556828,5.264232,8.497194,4901.0,2301,1.438646,1
TTTGTTGTCACAAGAA-1-1,16992-2D,1,mouse,1168,7.063904,2138.0,7.668094,31.571562,39.569691,49.906455,68.755847,173.0,5.159055,8.091675,7.667626,2138.0,1168,0.485150,0
TTTGTTGTCCTACGGG-1-1,16992-2D,1,mouse,1908,7.554335,4492.0,8.410275,31.923419,41.874443,51.246661,66.651825,348.0,5.855072,7.747106,8.410053,4492.0,1908,1.118800,2


In [8]:
adata = adata_read.copy()

In [9]:
adata.layers['counts'] = adata.X

Let's keep a copy of the full dataset for the subsequent analyses before we subset on highly variable genes:

In [10]:
adata_ = adata.copy()

In [11]:
sc.pp.highly_variable_genes(
    adata_,
    n_top_genes=16000,
    flavor="seurat_v3",
    subset=True,
    layer="counts"
)

In [12]:
adata.obs['group'] = adata.obs['louvain_r0.5'].astype('string')

In [14]:
# find DE genes by t-test
adata.uns['log1p']["base"] = None
sc.tl.rank_genes_groups(adata, 'group', method='t-test', key_added = "t-test")

### Get the Pathway and map it to mm

In [15]:
from pyorthomap import FindOrthologs 
hs2mm = FindOrthologs(
          host = 'http://www.ensembl.org',
          mart = 'ENSEMBL_MART_ENSEMBL',
          from_dataset = 'hsapiens_gene_ensembl',
          to_dataset = 'mmusculus_gene_ensembl',
          from_filters = 'hgnc_symbol',
          from_values = ['TP53', 'TERT'],
          to_attributes = ['external_gene_name'],
          to_homolog_attribute = 'mmusculus_homolog_ensembl_gene',
          from_gene_id_name = 'human_ensembl_gene_id',
          to_gene_id_name = 'mouse_ensembl_gene_id'
    )
    
hs2mm.map()

  0%|          | 0/1 [00:00<?, ?it/s]

,external_gene_name,mouse_ensembl_gene_id,human_ensembl_gene_id,hgnc_symbol
0,Tert,ENSMUSG00000021611,ENSG00000164362,TERT
1,Trp53,ENSMUSG00000059552,ENSG00000141510,TP53


### S1 Pathway

In [16]:
%%R
library(readxl)
my_pathways_s1=read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s1.xlsx')
my_pathways_s1=as.list(my_pathways_s1)

In [17]:
df = pd.read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s1.xlsx') # can also index sheet by name or fetch all sheets
hs_pathway_genes_s1 = np.array(df['Conserved_EMP_Signature'].tolist())
len(hs_pathway_genes_s1)

328

In [18]:
from pyorthomap import findOrthologsMmHs, findOrthologsHsMm
orthologies_hsmm = findOrthologsHsMm(from_filters = 'hgnc_symbol',
                  from_values = hs_pathway_genes_s1).map()
                  # from_values = [list(stats_df.index)]).map() # [stats.gene.names]).map()

  0%|          | 0/2 [00:00<?, ?it/s]

HTTPError: 414 Client Error: Request-URI Too Large for url: http://www.ensembl.org:80/biomart/martservice?query=%3CQuery+datasetConfigVersion%3D%220.6%22+formatter%3D%22TSV%22+header%3D%221%22+uniqueRows%3D%221%22+virtualSchemaName%3D%22default%22%3E%3CDataset+interface%3D%22default%22+name%3D%22mmusculus_gene_ensembl%22%3E%3CAttribute+name%3D%22external_gene_name%22+%2F%3E%3CAttribute+name%3D%22ensembl_gene_id%22+%2F%3E%3CFilter+name%3D%22link_ensembl_gene_id%22+value%3D%22nan%2Cnan%2CENSMUSG00000030342%2CENSMUSG00000046223%2CENSMUSG00000046733%2CENSMUSG00000022615%2Cnan%2CENSMUSG00000034708%2CENSMUSG00000000303%2CENSMUSG00000026864%2CENSMUSG00000003617%2CENSMUSG00000006519%2CENSMUSG00000019853%2CENSMUSG00000030800%2CENSMUSG00000026479%2CENSMUSG00000018171%2CENSMUSG00000022836%2CENSMUSG00000025064%2CENSMUSG00000000078%2CENSMUSG00000032294%2CENSMUSG00000051732%2CENSMUSG00000046173%2CENSMUSG00000015143%2CENSMUSG00000032373%2CENSMUSG00000029580%2CENSMUSG00000021270%2CENSMUSG00000031996%2

In [ ]:
my_pathways_s1 = orthologies_hsmm['external_gene_name'].tolist()
my_pathways_s1[:10]

In [ ]:
%R -i my_pathways_s1

## S2 Pathway

In [ ]:
%%R
library(readxl)
my_pathways_s2=read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s2.xlsx')
my_pathways_s2=as.list(my_pathways_s2)
# my_pathways

In [ ]:
df = pd.read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s2.xlsx') # can also index sheet by name or fetch all sheets
hs_pathway_genes_s2 = np.array(df['Malignant_specific_EMP'].tolist())
len(hs_pathway_genes_s2)

In [ ]:
from pyorthomap import findOrthologsMmHs, findOrthologsHsMm
orthologies_hsmm = findOrthologsHsMm(from_filters = 'hgnc_symbol',
                  from_values = hs_pathway_genes_s2).map()
                  # from_values = [list(stats_df.index)]).map() # [stats.gene.names]).map()

In [ ]:
my_pathways_s2 = orthologies_hsmm['external_gene_name'].tolist()
my_pathways_s2[:10]

In [ ]:
%R -i my_pathways_s2

## Hallmark EMP Pathway

In [ ]:
%%R
all_pathways <- gmtPathways('../data/Hallmark/h.all.v7.5.1.symbols.gmt')
my_pathways_hallmark_ <- all_pathways['HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION']

# Define the file name that will be deleted
fn <- '../data/EMT_gene_sets/hallmark_emt.csv'
# Check its existence
if (file.exists(fn)) {
  # Delete file if it exists
  file.remove(fn)
}

lapply(my_pathways_hallmark_, function(x) write.table( data.frame(x), fn, append=T, sep=','))
my_pathways_hallmark_

In [ ]:
hs_pathway_genes_hallmark = pd.read_csv('../data/EMT_gene_sets/hallmark_emt.csv')['x'].tolist()

In [ ]:
from pyorthomap import findOrthologsMmHs, findOrthologsHsMm
orthologies_hsmm = findOrthologsHsMm(from_filters = 'hgnc_symbol',
                  from_values = hs_pathway_genes_hallmark).map()
                  # from_values = [list(stats_df.index)]).map() # [stats.gene.names]).map()

In [ ]:
my_pathways_hallmark = orthologies_hsmm['external_gene_name'].tolist()
my_pathways_hallmark[:10]

In [ ]:
%R -i my_pathways_hallmark

### Cell-level pathway activity scoring using AUCell

In [ ]:
# Ignore this error. It occurs for some unknown reason, but adata is imported and you can continue to run the next cells
%R -i adata

In [ ]:
%%R
# Bioconductor 3.15 and R 4.2.0
# adata_aucell <- AUCell_run(adata_ , pathways)

cells_rankings <- AUCell_buildRankings(adata, plotStats=FALSE)
cells_AUC_s1 <- AUCell_calcAUC(my_pathways_s1, cells_rankings)
cells_AUC_s2 <- AUCell_calcAUC(my_pathways_s2, cells_rankings)
cells_AUC_hallmark <- AUCell_calcAUC(my_pathways_hallmark, cells_rankings)

In [ ]:
%%R
dim(cells_AUC_s1)

In [ ]:
%%R
dim(cells_AUC_s2)

In [ ]:
%%R
dim(cells_AUC_hallmark)

In [ ]:
%%R
# S1
aucell_scores_s1 <- data.frame(cells_AUC_s1@assays@data$AUC)
rownames(aucell_scores_s1) <- cells_AUC_s1@NAMES
colnames(aucell_scores_s1) <- colnames(adata)
# S2
aucell_scores_s2 <- data.frame(cells_AUC_s2@assays@data$AUC)
rownames(aucell_scores_s2) <- cells_AUC_s2@NAMES
colnames(aucell_scores_s2) <- colnames(adata)
# HALLMARK
aucell_scores_hallmark <- data.frame(cells_AUC_hallmark@assays@data$AUC)
rownames(aucell_scores_hallmark) <- cells_AUC_hallmark@NAMES
colnames(aucell_scores_hallmark) <- colnames(adata)

Export the results back to python 

In [ ]:
%%R -o aucell_scores_s1
aucell_scores_s1 = aucell_scores_s1

In [ ]:
%%R -o aucell_scores_s2 
aucell_scores_s2 = aucell_scores_s2

In [ ]:
%%R -o aucell_scores_hallmark 
aucell_scores_hallmark = aucell_scores_hallmark

In [ ]:
aucell_scores_s1 = aucell_scores_s1.T
aucell_scores_s2 = aucell_scores_s2.T
aucell_scores_hallmark = aucell_scores_hallmark.T

We now add the scores for the interferon-related REACTOME pathways to the anndata object and annotate the activity level of these pathways in each of the cells on the UMAP:

In [ ]:
EMP_pathways = ['geneSet']

adata.obs['EMP Pathway S1 Cook et. al.'] = aucell_scores_s1[EMP_pathways]
adata.obs['EMP Pathway S2 Cook et. al.'] = aucell_scores_s2[EMP_pathways]
adata.obs['EMP Pathway\nHALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION'] = aucell_scores_hallmark[EMP_pathways]

In [ ]:
adata.obs['Cluster'] = adata.obs['louvain_r0.5']

Plot the scores on the umap

In [ ]:
sc.pl.umap(
    adata,
    color=["Cluster"],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/16992-2D/emp/UMAP_OnlyCluster.pdf'
)
os.rename("figures/umap/16992-2D/emp/UMAP_OnlyCluster.pdf", 
          "../results/task3/16992-2D_emp_UMAP_OnlyCluster.pdf")

In [ ]:
sc.pl.umap(
    adata,
    color=["EMP Pathway S1 Cook et. al."],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/16992-2D/emp/UMAP_OnlyS1.pdf'
)
os.rename("figures/umap/16992-2D/emp/UMAP_OnlyS1.pdf", 
          "../results/task3/16992-2D_emp_UMAP_OnlyS1.pdf")

In [ ]:
sc.pl.umap(
    adata,
    color=["EMP Pathway S2 Cook et. al."],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/16992-2D/emp/UMAP_OnlyS2.pdf'
)
os.rename("figures/umap/16992-2D/emp/UMAP_OnlyS2.pdf", 
          "../results/task3/16992-2D_emp_UMAP_OnlyS2.pdf")

In [ ]:
sc.pl.umap(
    adata,
    color=["EMP Pathway\nHALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION"],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/16992-2D/emp/UMAP_OnlyHallmark.pdf'
)
os.rename("figures/umap/16992-2D/emp/UMAP_OnlyHallmark.pdf", 
          "../results/task3/16992-2D_emp_UMAP_OnlyHallmark.pdf")

### Violin

In [ ]:
sc.pl.violin(adata, 
             keys=["EMP Pathway S1 Cook et. al."],
             groupby='Cluster', 
             rotation=90,
             xlabel='Cluster',
             inner="quartile",
             wspace = 0.4,
             save='/16992-2D/emp_violin_counts_OnlyS1.pdf'
            )
os.rename("figures/violin/16992-2D/emp_violin_counts_OnlyS1.pdf", 
          "../results/task3/16992-2D_emp_violin_counts_OnlyS1.pdf")

In [ ]:
sc.pl.violin(adata, 
             keys=["EMP Pathway S2 Cook et. al."],
             groupby='Cluster', 
             rotation=90,
             xlabel='Cluster',
             inner="quartile",
             wspace = 0.4,
             save='/16992-2D/emp_violin_counts_OnlyS2.pdf'
            )
os.rename("figures/violin/16992-2D/emp_violin_counts_OnlyS2.pdf", 
          "../results/task3/16992-2D_emp_violin_counts_OnlyS2.pdf")

In [ ]:
sc.pl.violin(adata, 
             keys=["EMP Pathway\nHALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION"],
             groupby='Cluster', 
             rotation=90,
             xlabel='Cluster',
             inner="quartile",
             wspace = 0.4,
             save='/16992-2D/emp_violin_counts_OnlyHallmark.pdf'
            )
os.rename("figures/violin/16992-2D/emp_violin_counts_OnlyHallmark.pdf", 
          "../results/task3/16992-2D_emp_violin_counts_OnlyHallmark.pdf")